In [1]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import math
import pandas as pd
import datetime

In [2]:
ds = pd.read_excel('GeocodingAttractions.xlsx', sheet_name='Attractions', decimal=",", engine='openpyxl')
ds.describe(include='all', datetime_is_numeric=True).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,99,99,MAG001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Name,99,99,Sagrada Familia,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,99,24,Spain,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ISO,99,24,ES,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
location_lat,99.0,NaN,NaN,NaN,46.418104,6.675169,34.755986,41.106853,45.438977,51.330414,60.397435
location_long,99.0,NaN,NaN,NaN,8.712303,12.356655,-9.396203,-1.701258,6.183286,16.924632,37.62138
location_address,99,98,"Schloss Schönbrunn, 47, Schönbrunner Schlossst...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cluster,99.0,NaN,NaN,NaN,17.030303,11.282508,0.0,7.0,15.0,26.0,39.0


In [3]:
# We multiply our number per 10000 in order to use int instead of float, because otherwise the function validate dosen't work
points = ds[['location_lat','location_long']]
array = np.full((99, 2), 10000, dtype=int)
points = np.multiply(points,array)
points = points.astype(int) 
print(points)



    location_lat  location_long
0         414034          21744
1         414142          21524
2         515081           -762
3         413915          21646
4         500874         144209
..           ...            ...
94        437232         103957
95        394742          -3785
96        406564         -47075
97        486935          61832
98        437692         112548

[99 rows x 2 columns]


In [4]:
from typing import List, Tuple

def distance(origin: Tuple[float, float], destination: Tuple[float, float]) -> float:
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (
        math.sin(dlat / 2) * math.sin(dlat / 2)
        + math.cos(math.radians(lat1))
        * math.cos(math.radians(lat2))
        * math.sin(dlon / 2)
        * math.sin(dlon / 2)
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c
    return d

def create_clusters(
    number_of_clusters: int, points: List[Tuple[float, float]]
) -> np.ndarray:
    kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(points)
    l_array = np.array([[label] for label in kmeans.labels_])
    clusters = np.append(points, l_array, axis=1)
    return clusters

def validate_solution(max_dist: float, clusters: np.ndarray) -> bool:
    _, __, n_clust = clusters.max(axis=0)
    n_clust = int(n_clust)
    for i in range(n_clust):
        two_d_cluster = clusters[clusters[:, 2] == i][:, np.array([True, True, False])]
        if not validate_cluster(max_dist, two_d_cluster):
            return False
        else:
            continue
    return True

def validate_cluster(max_dist: float, cluster: np.ndarray) -> bool:
    distances = cdist(
        cluster, cluster, lambda ori, des: int(round(distance(ori, des)))
    )
    print(distances)
    print(30 * "-")
    for item in distances.flatten():
        if item > max_dist:
            return False
    return True


In [5]:

# we don't need the validation here
for i in range(2,len(points)):
    print(i)
    print(validate_solution(30,create_clusters(i,points)))

2
[[    0. 11439.  3775. ...  6226. 10268. 19164.]
 [11439.     0.  7673. ...  5237. 13273.  8846.]
 [ 3775.  7673.     0. ...  2452. 11863. 16474.]
 ...
 [ 6226.  5237.  2452. ...     0. 12540. 14079.]
 [10268. 13273. 11863. ... 12540.     0. 10311.]
 [19164.  8846. 16474. ... 14079. 10311.     0.]]
------------------------------
False
3
[[    0. 14767. 12788.  9976.  8396. 11327. 12742. 13448. 12677. 14607.]
 [14767.     0.  8599.  5204.  6372. 12225.  9289.  6039. 12288.  2484.]
 [12788.  8599.     0. 12646. 11385.  4069.   901.  2791.  6354.  6223.]
 [ 9976.  5204. 12646.     0.  2628. 16706. 13531.  9882. 17376.  6864.]
 [ 8396.  6372. 11385.  2628.     0. 14958. 12268.  8964. 17717.  7002.]
 [11327. 12225.  4069. 16706. 14958.     0.  3197.  6851.  3173. 10101.]
 [12742.  9289.   901. 13531. 12268.  3197.     0.  3654.  5484.  6988.]
 [13448.  6039.  2791.  9882.  8964.  6851.  3654.     0.  8763.  3567.]
 [12677. 12288.  6354. 17376. 17717.  3173.  5484.  8763.     0. 11052.]
 [

In [6]:
ds['cluster'] = create_clusters(40,points)[:,2]
ds.head()

,ID,Name,Country,ISO,location_lat,location_long,location_address,cluster
0,MAG001,Sagrada Familia,Spain,ES,41.403479,2.174410,"Basílica de la Sagrada Família, 401, Carrer de...",1
1,MAG002,Parc Guell,Spain,ES,41.414235,2.152458,"Park Güell, Carrer d'Olot, la Salut, Gràcia, B...",1
2,MAG003,Tower of London,England,UK,51.508120,-0.076222,"Tower of London, Tower Hill, Tower Liberty, Wh...",13
3,MAG004,Casa Batllo,Spain,ES,41.391545,2.164696,"Casa Batlló, 43, Passeig de Gràcia, la Dreta d...",1
4,MAG005,Staromestske namesti,Czech Republic,CZ,50.087455,14.420967,"Staroměstské náměstí, Staré Město, Hlavní měst...",38


In [110]:
# dsNoOutlier.to_excel('cleaned.xlsx', index=False, sheet_name='Reviews')
ds.to_excel('GeocodingAttractions.xlsx', index=False, sheet_name='Attractions')

print(datetime.datetime.now())

2022-12-09 18:32:20.888297
